In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/vnp/

/content/drive/MyDrive/vnp


In [3]:
%pwd

'/content/drive/MyDrive/vnp'

In [4]:
!git clone https://github.com/THU-MIG/yolov10.git

Cloning into 'yolov10'...
remote: Enumerating objects: 20329, done.
remote: Counting objects: 100% (1527/1527), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 20329 (delta 1450), reused 1363 (delta 1351), pack-reused 18802
Receiving objects: 100% (20329/20329), 11.19 MiB | 11.12 MiB/s, done.
Resolving deltas: 100% (14326/14326), done.
Updating files: 100% (580/580), done.


In [4]:
!pip install /content/drive/MyDrive/vnp/yolov10/.

Processing ./yolov10
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1

In [5]:
!pip install easyocr pandas google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 19.1 MB/s eta 0:00:00


In [6]:
import pandas as pd
from datetime import datetime
from ultralytics import YOLOv10
import easyocr
import cv2
from google.cloud import bigquery
import os

In [7]:
service_account_path = '/content/drive/MyDrive/vnp/yolov10/oauth-credentials.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_path

client = bigquery.Client()

In [8]:
def vehicle_data():
  query = """SELECT * FROM `deep-ethos-427902-u1.vehicle_number_plate.Vehicles`"""
  query_job = client.query(query)
  vehicle_data_df = query_job.result().to_dataframe()
  return vehicle_data_df

In [9]:
def entry_data():
  query = """SELECT * FROM `deep-ethos-427902-u1.vehicle_number_plate.Vehicle_Entry`"""
  query_job = client.query(query)
  entry_data_df = query_job.result().to_dataframe()
  return entry_data_df

In [10]:
vehicle_data_df = vehicle_data()

In [11]:
model = YOLOv10("/content/drive/MyDrive/vnp/yolov10/best.pt")
ocr = easyocr.Reader(['en'], gpu=True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [12]:
def ocr_on_image(img_path, cord_img):
  xmin = int(cord_img[0][0])
  ymin = int(cord_img[0][1])
  xmax = int(cord_img[0][2])
  ymax = int(cord_img[0][3])
  cropped_img = cv2.imread(img_path,cv2.IMREAD_COLOR)[ymin:ymax,xmin:xmax]
  gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2GRAY)
  ocr_result = ocr.readtext(gray_img)
  if ocr_result == []:
    return "No Text found"
  else:
    return ocr_result[0][1]

In [13]:
def vehicle_number_predictor(img_path):
  results = model.predict(source=img_path)
  cord_img = results[0].boxes.xyxy
  return ocr_on_image(img_path, cord_img)

In [14]:
predict = vehicle_number_predictor("/content/drive/MyDrive/vnp/yolov10/KL16.jpg")
time = datetime.now()


image 1/1 /content/drive/MyDrive/vnp/yolov10/KL16.jpg: 640x480 1 number_plate, 91.8ms
Speed: 12.0ms preprocess, 91.8ms inference, 365.0ms postprocess per image at shape (1, 3, 640, 480)


In [15]:
predict

'KL 4304444'

In [27]:
def no_space(string):
    return string.replace(" ","")

In [35]:
def insert_into_vehicle_data(number_plate, vehicle_data):
    table = client.dataset('vehicle_number_plate').table('Vehicles')
    if vehicle_data['vehicle_id'].shape[0] == 0:
      vehicle_id = 1
    else:
      vehicle_id = int(max(vehicle_data['vehicle_id'])+1)
    row_to_insert = [{"vehicle_id": vehicle_id, "vehicle_number": number_plate,"vehicle_type":"Guest"}]
    errors = client.insert_rows_json(table, row_to_insert)
    if errors:
      print("Errors occurred while inserting rows:", errors)
    else:
      print("Vehicles Data updated successfully")

In [36]:
def insert_into_vehicle_entry(number_plate, time, vehicle_data):
    vehicle_id = vehicle_data[vehicle_data['vehicle_number'] == number_plate]['vehicle_id']
    vehicle_id = int(vehicle_id.iloc[0])
    entry_data_df = entry_data()
    entry_time_str = time.strftime('%Y-%m-%d %H:%M:%S')
    if entry_data_df['entry_id'].shape[0] == 0:
      entry_id = 1
    else:
      entry_id = int(max(entry_data_df['entry_id'])+1)
    table = client.dataset('vehicle_number_plate').table('Vehicle_Entry')
    row_to_insert = [{"entry_id":entry_id, "vehicle_id": vehicle_id,"entry_time":entry_time_str}]
    errors = client.insert_rows_json(table, row_to_insert)
    if errors:
      print("Errors occurred while inserting rows:", errors)
    else:
      print("Vehicles Data updated successfully")

In [37]:
def vehicle_add(vehicle_data, predict, time):
    number_plate = no_space(predict)
    vehicle_data['vehicle_number'] = vehicle_data['vehicle_number'].apply(no_space)
    if vehicle_data[vehicle_data['vehicle_number'] == number_plate].shape[0] == 0:
        insert_into_vehicle_data(number_plate, vehicle_data)
        vehicle_data_df = vehicle_data()
        vehicle_data['vehicle_number'] = vehicle_data['vehicle_number'].apply(no_space)
        insert_into_vehicle_entry(number_plate,time,vehicle_data)
    else:
        insert_into_vehicle_entry(number_plate,time,vehicle_data)


In [38]:
vehicle_add(vehicle_data_df, predict, time)

Vehicles Data updated successfully
